In [83]:
import pandas as pd

In [9]:
import requests
from bs4 import BeautifulSoup

def scrape_hot100():
    url = "https://www.billboard.com/charts/hot-100/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    song_elements = soup("div", class_="o-chart-results-list-row-container")
    #print("song_elements:", song_elements)
    
    songs = []
    for element in song_elements:
        song = {}
        song["title"] = element.find("h3").text.strip()
        song["artist"] = element.find("h3").find_next("span").text.strip()
        songs.append(song)
    
    #print("songs:", songs)
    
    hot100_df = pd.DataFrame(songs)
    return hot100_df

    # Save the DataFrame to a CSV file
    hot100_df.to_csv('hot100_songs.csv', index=False)

hot100_df = scrape_hot100()
hot100_df.head()

,title,artist
0,Kill Bill,SZA
1,Last Night,Morgan Wallen
2,Flowers,Miley Cyrus
3,Princess Diana,Ice Spice & Nicki Minaj
4,Ella Baila Sola,Eslabon Armado X Peso Pluma


In [36]:
# load the data
hot100_df = pd.read_csv('../data/hot100_songs.csv')

In [ ]:
from tinydb import TinyDB, Query
db = TinyDB('hot100.json')
db.insert_multiple(hot100_df.to_dict('records'))

In [53]:
# rename column "song" to "title"
hot100_df = hot100_df.rename(columns={'Song': 'title'})
hot100_df = hot100_df.rename(columns={'Artist': 'artists'})
# check the db in a dataframe
hot100_df['hot'] = 1
# add a column "identifier" by concatenating "title" and "artist" 
hot100_df['identifier'] = hot100_df['title'] + ' - ' + hot100_df['artists']
hot100_df['identifier'] = hot100_df['identifier'].str.lower()
hot100_df.head()

,title,artists,hot,identifier
0,Last Night,Morgan Wallen,1,last night - morgan wallen
1,Flowers,Miley Cyrus,1,flowers - miley cyrus
2,Princess Diana,Ice Spice & Nicki Minaj,1,princess diana - ice spice & nicki minaj
3,Ella Baila Sola,Eslabon Armado X Peso Pluma,1,ella baila sola - eslabon armado x peso pluma
4,Creepin',"Metro Boomin, The Weeknd & 21 Savage",1,"creepin' - metro boomin, the weeknd & 21 savage"


In [41]:
# load the genres data
features = pd.read_csv('../data/tracks_features.csv')
# show all columns 
pd.set_option('display.max_columns', None)
features.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [82]:
# value unique for key 
features['key'].value_counts()

7     155382
0     149568
2     144583
9     133270
5     100006
4      97612
1      96357
11     81456
10     77164
8      65348
6      63502
3      39777
Name: key, dtype: int64

In [68]:
# keep only name and artists columns
nothot_df = features[['name', 'artists', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness' ,'valence', 'tempo', 'duration_ms', 'time_signature', 'year']]
nothot_df.rename(columns={'name': 'title'}, inplace=True)
# keep first artist in artists column
nothot_df['artists'] = nothot_df['artists'].str.split(',').str[0]
# remove [' and '] from artists column 
nothot_df['artists'] = nothot_df['artists'].str.replace(r"\['|'\]", '', regex=True)

nothot_df['identifier'] = nothot_df['title'] + ' - ' + nothot_df['artists']
# lowercase all identifier column 
nothot_df['identifier'] = nothot_df['identifier'].str.lower()
nothot_df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_9212\2452074460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nothot_df.rename(columns={'name': 'title'}, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_9212\2452074460.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nothot_df['artists'] = nothot_df['artists'].str.split(',').str[0]
C:\Users\user\AppData\Local\Temp\ipykernel_9212\2452074460.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,title,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,identifier
0,Testify,Rage Against The Machine,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,testify - rage against the machine
1,Guerrilla Radio,Rage Against The Machine,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,guerrilla radio - rage against the machine
2,Calm Like a Bomb,Rage Against The Machine,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,calm like a bomb - rage against the machine
3,Mic Check,Rage Against The Machine,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,mic check - rage against the machine
4,Sleep Now In the Fire,Rage Against The Machine,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,sleep now in the fire - rage against the machine


In [74]:
# join "nothot" with "hot100_df" dataframes (join on 'title' and the 'name') 

recommend = nothot_df.merge(hot100_df[['identifier', 'hot']], on='identifier', how='left')
recommend['hot'] = recommend['hot'].fillna(0)

recommend.head()


,title,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,identifier,hot
0,Testify,Rage Against The Machine,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,testify - rage against the machine,0.0
1,Guerrilla Radio,Rage Against The Machine,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,guerrilla radio - rage against the machine,0.0
2,Calm Like a Bomb,Rage Against The Machine,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,calm like a bomb - rage against the machine,0.0
3,Mic Check,Rage Against The Machine,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,mic check - rage against the machine,0.0
4,Sleep Now In the Fire,Rage Against The Machine,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,sleep now in the fire - rage against the machine,0.0


In [70]:
# filter on "hot" == 1 
recommend = recommend[recommend['hot'] == 1]
recommend.head(120)

,title,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,identifier,hot
967129,Under The Influence,Chris Brown,0.733,0.690,9,-5.529,0,0.0427,0.0635,0.000001,0.105,0.310,116.992,184613,4.0,2019,under the influence - chris brown,1.0
967919,Under The Influence,Chris Brown,0.727,0.704,9,-5.534,0,0.0426,0.0524,0.000002,0.102,0.318,116.987,184613,4.0,2019,under the influence - chris brown,1.0
